In [ ]:
%matplotlib inline
from itertools import izip

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from keras.engine import Input, Model, merge
from keras.layers import BatchNormalization, UpSampling2D, Dropout, Flatten, Dense, LeakyReLU, Activation
from keras.layers.convolutional import Convolution2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2


np.random.seed(1337)  # for reproducibility


# Read files using a Generator... You have to download your needed Dataset
# The available datasets are mentioned in the paper
# This function is made to fetch the "CMP Building Facades" dataset from your HardDrive
def readFilesByGenerator(batchSize=1, targetSize=(256, 256)):
    print("Reading Images...")

    train_gray_datagen = ImageDataGenerator(rescale=1. / 255)
    train_rgb_datagen = ImageDataGenerator(rescale=1. / 255)
    test_gray_datagen = ImageDataGenerator(rescale=1. / 255)
    test_rgb_datagen = ImageDataGenerator(rescale=1. / 255)

    train_gray_generator = train_gray_datagen.flow_from_directory(
        './Data/Train',
        batch_size=batchSize,
        target_size=targetSize,
        color_mode='rgb',
        class_mode=None,
        shuffle=True,
        seed=1,
        classes=['Facade'])
    train_rgb_generator = train_rgb_datagen.flow_from_directory(
        './Data/Train',
        batch_size=batchSize,
        target_size=targetSize,
        color_mode='rgb',
        class_mode=None,
        shuffle=True,
        seed=1,
        classes=['Real'])

    test_gray_generator = test_gray_datagen.flow_from_directory(
        './Data/Test',
        batch_size=batchSize,
        target_size=targetSize,
        color_mode='rgb',
        class_mode=None,
        shuffle=True,
        seed=2,
        classes=['Facade'])

    test_rgb_generator = test_rgb_datagen.flow_from_directory(
        './Data/Test',
        batch_size=batchSize,
        target_size=targetSize,
        color_mode='rgb',
        class_mode=None,
        shuffle=True,
        seed=2,
        classes=['Real'])

    # Iterate through them using a lazy izip because of Memory constraints
    train_generator = izip(train_gray_generator, train_rgb_generator)
    test_generator = izip(test_gray_generator, test_rgb_generator)

    print("Finished Reading Images...")
    return train_generator, test_generator


# Create the Generator (This is the U-Net mentioned in the paper)
def generator_model():
    generator_input = Input(batch_shape=(None, 3, 256, 256), name='generator_input')

    # ENCODER
    conv1_64 = Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same', W_regularizer=l2(0.001))(generator_input)
    conv1_64 = LeakyReLU(alpha=0.2)(conv1_64)

    conv2_128 = Convolution2D(128, 4, 4, subsample=(2, 2), border_mode='same', W_regularizer=l2(0.001))(conv1_64)
    conv2_128 = BatchNormalization(axis=1, mode=2)(conv2_128)
    conv2_128 = LeakyReLU(alpha=0.2)(conv2_128)

    conv3_256 = Convolution2D(256, 4, 4, subsample=(2, 2), border_mode='same', W_regularizer=l2(0.001))(conv2_128)
    conv3_256 = BatchNormalization(axis=1, mode=2)(conv3_256)
    conv3_256 = LeakyReLU(alpha=0.2)(conv3_256)

    conv4_512 = Convolution2D(512, 4, 4, subsample=(2, 2), border_mode='same', W_regularizer=l2(0.001))(conv3_256)
    conv4_512 = BatchNormalization(axis=1, mode=2)(conv4_512)
    conv4_512 = LeakyReLU(alpha=0.2)(conv4_512)

    conv5_512 = Convolution2D(512, 4, 4, subsample=(2, 2), border_mode='same', W_regularizer=l2(0.001))(conv4_512)
    conv5_512 = BatchNormalization(axis=1, mode=2)(conv5_512)
    conv5_512 = LeakyReLU(alpha=0.2)(conv5_512)

    conv6_512 = Convolution2D(512, 4, 4, subsample=(2, 2), border_mode='same', W_regularizer=l2(0.001))(conv5_512)
    conv6_512 = BatchNormalization(axis=1, mode=2)(conv6_512)
    conv6_512 = LeakyReLU(alpha=0.2)(conv6_512)

    conv7_512 = Convolution2D(512, 4, 4, subsample=(2, 2), border_mode='same', W_regularizer=l2(0.001))(conv6_512)
    conv7_512 = BatchNormalization(axis=1, mode=2)(conv7_512)
    conv7_512 = LeakyReLU(alpha=0.2)(conv7_512)

    conv8_512 = Convolution2D(512, 4, 4, subsample=(2, 2), border_mode='same', W_regularizer=l2(0.001))(conv7_512)
    conv8_512 = BatchNormalization(axis=1, mode=2)(conv8_512)
    conv8_512 = LeakyReLU(alpha=0.2)(conv8_512)

    # DECODER
    conv9_512 = UpSampling2D()(conv8_512)
    conv9_512 = Convolution2D(512, 4, 4, border_mode='same', W_regularizer=l2(0.001))(conv9_512)
    conv9_512 = BatchNormalization(axis=1, mode=2)(conv9_512)
    conv9_512 = Dropout(0.5)(conv9_512)
    conv9_512 = Activation('relu')(conv9_512)
    merge_2_2 = merge((conv9_512, conv7_512), mode='concat', concat_axis=1)

    conv10_512 = UpSampling2D()(merge_2_2)
    conv10_512 = Convolution2D(512, 4, 4, border_mode='same', W_regularizer=l2(0.001))(conv10_512)
    conv10_512 = BatchNormalization(axis=1, mode=2)(conv10_512)
    conv10_512 = Dropout(0.5)(conv10_512)
    conv10_512 = Activation('relu')(conv10_512)
    merge_4_4 = merge((conv10_512, conv6_512), mode='concat', concat_axis=1)

    conv11_512 = UpSampling2D()(merge_4_4)
    conv11_512 = Convolution2D(512, 4, 4, border_mode='same', W_regularizer=l2(0.001))(conv11_512)
    conv11_512 = BatchNormalization(axis=1, mode=2)(conv11_512)
    conv11_512 = Dropout(0.5)(conv11_512)
    conv11_512 = Activation('relu')(conv11_512)
    merge_8_8 = merge((conv11_512, conv5_512), mode='concat', concat_axis=1)

    conv12_512 = UpSampling2D()(merge_8_8)
    conv12_512 = Convolution2D(512, 4, 4, border_mode='same', W_regularizer=l2(0.001))(conv12_512)
    conv12_512 = BatchNormalization(axis=1, mode=2)(conv12_512)
    conv12_512 = Activation('relu')(conv12_512)
    merge_16_16 = merge((conv12_512, conv4_512), mode='concat', concat_axis=1)

    conv13_512 = UpSampling2D()(merge_16_16)
    conv13_512 = Convolution2D(512, 4, 4, border_mode='same', W_regularizer=l2(0.001))(conv13_512)
    conv13_512 = BatchNormalization(axis=1, mode=2)(conv13_512)
    conv13_512 = Activation('relu')(conv13_512)
    merge_32_32 = merge((conv13_512, conv3_256), mode='concat', concat_axis=1)

    conv14_256 = UpSampling2D()(merge_32_32)
    conv14_256 = Convolution2D(512, 4, 4, border_mode='same', W_regularizer=l2(0.001))(conv14_256)
    conv14_256 = BatchNormalization(axis=1, mode=2)(conv14_256)
    conv14_256 = Activation('relu')(conv14_256)
    merge_64_64 = merge((conv14_256, conv2_128), mode='concat', concat_axis=1)

    conv15_128 = UpSampling2D()(merge_64_64)
    conv15_128 = Convolution2D(256, 4, 4, border_mode='same', W_regularizer=l2(0.001))(conv15_128)
    conv15_128 = BatchNormalization(axis=1, mode=2)(conv15_128)
    conv15_128 = Activation('relu')(conv15_128)
    merge_128_128 = merge((conv15_128, conv1_64), mode='concat', concat_axis=1)

    conv16_64 = UpSampling2D()(merge_128_128)
    conv16_64 = Convolution2D(128, 4, 4, border_mode='same', W_regularizer=l2(0.001))(conv16_64)
    conv16_64 = BatchNormalization(axis=1, mode=2)(conv16_64)
    conv16_64 = Activation('relu')(conv16_64)
    merge_256_256 = merge((conv16_64, generator_input), mode='concat', concat_axis=1)

    generator_output = Convolution2D(3, 4, 4, activation='sigmoid', border_mode='same', name='generator_output')(merge_256_256)

    model = Model(input=generator_input, output=generator_output)

    return model


# Create the Discriminator (This is the 70x70 Receptive Field Discriminator mentioned in the paper)
def discriminator_model():
    discriminator_input = Input(batch_shape=(None, 3, 256, 256))

    conv1_64 = Convolution2D(64, 4, 4, subsample=(2, 2), W_regularizer=l2(0.001))(discriminator_input)
    conv1_64 = LeakyReLU(alpha=0.2)(conv1_64)

    conv2_128 = Convolution2D(128, 4, 4, subsample=(2, 2), W_regularizer=l2(0.001))(conv1_64)
    conv2_128 = BatchNormalization(axis=1, mode=2)(conv2_128)
    conv2_128 = LeakyReLU(alpha=0.2)(conv2_128)

    conv3_256 = Convolution2D(256, 4, 4, subsample=(2, 2), W_regularizer=l2(0.001))(conv2_128)
    conv3_256 = BatchNormalization(axis=1, mode=2)(conv3_256)
    conv3_256 = LeakyReLU(alpha=0.2)(conv3_256)

    conv4_512 = Convolution2D(512, 4, 4, subsample=(2, 2), W_regularizer=l2(0.001))(conv3_256)
    conv4_512 = BatchNormalization(axis=1, mode=2)(conv4_512)
    conv4_512 = LeakyReLU(alpha=0.2)(conv4_512)

    # conv5_512 = Convolution2D(512, 4, 4, subsample=(2, 2), W_regularizer=l2(0.001))(conv4_512)
    # conv5_512 = BatchNormalization(axis=1, mode=2)(conv5_512)
    # conv5_512 = LeakyReLU(alpha=0.2)(conv5_512)
    #
    # conv6_512 = Convolution2D(512, 4, 4, subsample=(2, 2), W_regularizer=l2(0.001))(conv5_512)
    # conv6_512 = BatchNormalization(axis=1, mode=2)(conv6_512)
    # conv6_512 = LeakyReLU(alpha=0.2)(conv6_512)

    flatten = Flatten()(conv4_512)
    discriminator_output = Dense(1, activation='sigmoid')(flatten)

    model = Model(input=discriminator_input, output=[discriminator_output, discriminator_input])

    return model


# Concat the two Networks and make the cGAN network
def generator_to_discriminator_model(generator, discriminator):
    discriminator.trainable = False
    generator_to_discriminator = Model(input=generator.input, output=discriminator(generator.output))

    return generator_to_discriminator


generator = generator_model()
# generator.load_weights('./Model/Archive/FinalizedModels/Facades/GeneratorWeights-Index1000')

discriminator = discriminator_model()
# discriminator.load_weights('./Model/Archive/FinalizedModels/Facades/DiscriminatorWeights-Index1000')

# Only train the Discriminator on the binary_crossentropy loss as the L1 loss isn't needed here
discriminator.compile(loss=['binary_crossentropy', 'mae'], loss_weights=[1., 0], optimizer=Adam(lr=0.00005, beta_1=0.5))

generator_to_discriminator = generator_to_discriminator_model(generator, discriminator)

# Train the cGAN with 2 losses, the binary_crossentropy loss counts for GAN and the L1 MAE loss has a weight of 100
generator_to_discriminator.compile(loss=['binary_crossentropy', 'mae'], loss_weights=[1., 100.], optimizer=Adam(lr=0.0002, beta_1=0.5))

BATCH_SIZE = 1
IMAGE_COUNT = 400
train_generator, test_generator = readFilesByGenerator(batchSize=BATCH_SIZE, targetSize=(256, 256))

d_loss_log = []
g_loss_log = []

for index, (train_data, test_data) in enumerate(izip(train_generator, test_generator)):
    X_train_gray = train_data[0]
    X_train_rgb = train_data[1]
    X_test_gray = test_data[0]
    X_test_rgb = test_data[1]

    # only run this once in the first epoch
    if index == 0:
        generated_train_images = generator.predict(X_train_gray)
        generated_test_images = generator.predict(X_test_gray)

    # Train the discriminator on real and fake data
    y = [1] * BATCH_SIZE
    d_loss_real = discriminator.train_on_batch(X_train_rgb, [np.array(y), X_train_rgb])[1]
    y = [0] * BATCH_SIZE
    d_loss_fake = discriminator.train_on_batch(generated_train_images, [np.array(y), X_train_rgb])[1]
    d_loss = (d_loss_real + d_loss_fake) / 2.0
    d_loss_log.append(d_loss)

    # Train the cGAN network on fake data
    discriminator.trainable = False
    g_loss = generator_to_discriminator.train_on_batch(X_train_gray, [np.array([1] * BATCH_SIZE), X_train_rgb])[1]
    g_loss_log.append(g_loss)
    discriminator.trainable = True

    # Generate images to plot and show
    generated_train_images = generator.predict(X_train_gray)
    generated_test_images = generator.predict(X_test_gray)
    test_accuracy = discriminator.predict(generated_test_images)[0]
    if index % 50 == 0 or (0.9 <= test_accuracy):
        print("Index %d D-Loss %.3f DG-Loss %.3f" % (index, d_loss, g_loss))

        fig = plt.figure(figsize=(50, 20))
        for imageIndex, image in enumerate(generated_test_images):
            ax = fig.add_subplot(3, len(generated_test_images), imageIndex + 1)
            ax.imshow(X_test_rgb[imageIndex].transpose(1, 2, 0))
            ax.axis("off")
            ax = fig.add_subplot(3, len(generated_test_images), len(generated_test_images) + imageIndex + 1)
            ax.imshow(X_test_gray[imageIndex].transpose(1, 2, 0))
            ax.axis("off")
            ax = fig.add_subplot(3, len(generated_test_images), len(generated_test_images) * 2 + imageIndex + 1)
            ax.imshow(generated_test_images[imageIndex].transpose(1, 2, 0))
            ax.axis("off")
        plt.subplots_adjust(bottom=0.6, wspace=0.05, hspace=0)
        fig.tight_layout()
        plt.savefig("./Model/Images/Index %d Accuracy %.3f.png" % (index, test_accuracy))
        print("Index %d Accuracy %.3f" % (index, test_accuracy))
        if 0.9 <= test_accuracy:
            plt.show()
        plt.close()

        plt.plot(d_loss_log, color='blue', label='D-Loss Total')
        plt.plot(g_loss_log, color='red', label='DG-Loss', alpha=0.75)
        plt.legend(loc='best')
        plt.ylim((0, 3))

        plt.show()
        plt.close()

    # Save the weights
    if (index != 0) and (index % 1000 == 0):
        generator.save_weights('./Model/Images/Models/GeneratorWeights-Index%s' % index, True)
        discriminator.save_weights('./Model/Images/Models/DiscriminatorWeights-Index%s' % index, True)
        print("Models Saved => Index %s" % index)

    # Clear the output as it tends to get crowded!
    if index != 0 and index % 1000 == 0:
        clear_output()
